In [12]:
import torch
from pathlib import Path
import sys
sys.path.append("../src")

from dataset import ShakespearDataset
from torch.utils.data import DataLoader, random_split

import torch.nn as nn
from torch.nn import functional as F

In [4]:
data_root_path = Path("../data")
data_root_path

WindowsPath('../data')

In [5]:
text = (data_root_path/"shakespear.txt").open("r").read()
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
chars = sorted(set(text))
vocab_size = len(chars)
vocab_size

65

In [7]:
ctoi = {c: i for i, c in enumerate(chars)}
itoc = {i: c for c, i in ctoi.items()}

encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join([itoc[i] for i in l])

print(encode("Hi there"))
print(decode(encode("Hi there")))

[20, 47, 1, 58, 46, 43, 56, 43]
Hi there


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [9]:
dataset = ShakespearDataset(data_root_path/"shakespear.txt")
dataset

Dataset(self.context_size=8)

In [10]:
dataset[0]

(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [11]:
train_size = int(len(dataset) * 0.7)
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)

In [13]:
emb_table = nn.Embedding(vocab_size, vocab_size)
emb_table

Embedding(65, 65)

In [14]:
for x, y in train_dataloader:
    break

x.shape, y.shape

(torch.Size([64, 8]), torch.Size([64, 8]))

In [16]:
logits = emb_table(x)
logits.shape

torch.Size([64, 8, 65])

In [17]:
logits = logits.reshape(-1, vocab_size)
logits.shape

torch.Size([512, 65])

In [19]:
y = y.reshape(-1)
y.shape

torch.Size([512])

In [30]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) 
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(x, y)
loss

tensor(4.7322, grad_fn=<NllLossBackward0>)

In [36]:
logits, _ = model(torch.ones((1, 1), dtype=torch.long))
logits.shape

torch.Size([1, 1, 65])

In [50]:
decode(model.generate(idx=torch.ones((1, 1), dtype=torch.long), max_new_tokens=100).tolist()[0])

" woiDqWJj!mjUh&XEMWMCTfj-eGDZIVlCo3NZT,&uu,T'wVnYQa'rDsK ULZjR.L\ngAyBRMD$MXe;k a;ki!m-pMHcGsJPKz,'ZDx"

In [61]:
chars[1]

' '